# Models
## KNN
Lets first create a K-NN model. During "training" it saves all dataframes in an internal storrage
During testing, it calculates the distance between the input and all stored samples and selects the (majority vote) label of the K nearest neighbors.

A good K can be found by using k-folds cross validation

In [1]:
import numpy as np
import pandas as pd
import math
from src.preprocess_data import preprocess_dataset, pca_dataset, balance_class_distribution
import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
import torch
from torch.utils.data import DataLoader, TensorDataset
from collections import deque

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
class KNN():
    MAX_MATRIX_ENTRIES = 2e8

    def __init__(self, num_cols: int, k: int) -> None:
        """Creates an empty KNN classifier

        Args:
            num_cols (int): The number of columns (features) the data is going to have, including the label.
            k (int): The number of neighbours that should be considered.
        """
        self.data = np.empty([0, num_cols])
        self.shape = self.data.shape
        self.k = k
        self.__verbose = False
        self.use_dims = None
        self.__pc = None
        return

    def __len__(self) -> int:
        return len(self.data)

    def __call__(self, *args, **kwds):
        return self.top_k(*args, **kwds)

    def verbose(self, verbose=True) -> None:
        """Sets the model to the verbose mode, where it comments on what it is doing.

        Args:
            verbose (bool, optional): Defaults to True.
        """
        self.__verbose = True
        return None

    def set_reduce_dimensions(self, num_leading_dim: int) -> None:
        """Sets KNN to use PCA in order to reduce the dimensionality of the data.

        Args:
            num_leading_dim (int): The number of PCs that should be used for the trasnformation.
        """
        self.use_dims = num_leading_dim
        return

    def get_pca(self):
        if self.use_dims is None:
            raise Exception("PCA can only be performed after setting a number of dimensions for reduction. Use ```KNN.set_reduce_dimensions```")
        if self.__pc is None:
            self.__pca(None)
        return self.__pc.components_
    
    def store_data(self, df: pd.DataFrame) -> None:
        """Stores a single dataframe into the internal storage

        Args:
            df (pd.DataFrame): The data to be saved.
        """
        self.data = np.concatenate([self.data, df.to_numpy()])
        self.shape = self.data.shape

        if self.__verbose:
            print("New shape is {}.".format(self.shape))
        # Reset internal
        self.__dat = None
        self.__lab = None
        self.__pc = None
        return

    def set_k(self, k: int) -> None:
        """Set another k hyperparameter

        Args:
            k (int): The new k
        """
        self.k = k

    def __pca(self, data: np.ndarray) -> np.ndarray:
        """Performs PCA on the internal saved data to calculate the transformed data"""
        # Check if saved PCs are available
        
        if self.__pc is None:
            # Calculate PCs
            self.__pc = PCA(n_components = self.use_dims).fit(self.data[:,:-1])
            
            self.data = np.concatenate([self.__pc.transform(self.data[:,:-1]), self.data[:,-1, None] ], axis = 1)
            self.shape = self.data.shape
            
        if data is None:
            return   
        
        # Transform data
        data = self.__pc.transform(data)
    
        return data

    def top_k(self, datapoint: np.ndarray) -> np.ndarray:
        """Calculates the label using the cosine similarity between each datapoint and the stored data points

        Args:
            datapoint (np.narray): An array of shape [N,C], where N is the number of datasamples (rows) and C the number of features (columns), not including the label

        Returns:
            np.ndarray: An array of shape [N], containing the label prediction 
        """
        

        datapoint = datapoint.copy()

        if self.use_dims is not None:
            datapoint = self.__pca(datapoint)
        
        assert datapoint.shape[1] == self.shape[1]-1
        
        
        # Pepare internal data representation for calculation
        if self.__dat == None:

            # Remove label column
            self.__dat = self.data[:, :-1]
            self.__lab = self.data[:, -1].astype(int)

            # Normalize each row to length 1
            self.__dat = self.__dat / \
                np.linalg.norm(self.__dat, axis=1, keepdims=True)

        datapoint = datapoint / \
            np.linalg.norm(datapoint, axis=1, keepdims=True)

        # Split input array into smaller ones if N*n is to big
        step_size = max(1, int(KNN.MAX_MATRIX_ENTRIES // len(self)))
        labels_accumulator = []

        max_i = math.ceil(len(datapoint) / step_size)

        if self.__verbose:
            print("Starts calculating on self.data ({}) and input ({}).".format(
                self.shape, datapoint.shape))
            _st_t = datetime.datetime.now()

        for i in range(max_i):

            if self.__verbose:
                _en_t = datetime.datetime.now()
                _t = _en_t - _st_t
                if i == 0:
                    _t_ex = datetime.timedelta(seconds=0)
                else:
                    _t_ex = _t * (max_i/i)

                print("Finsihed datapoints {}/{} ({}/{})    {}m{}s/{}m{}s".format(
                    i * step_size, len(datapoint),
                    i, max_i,
                    _t.seconds//60, _t.seconds % 60,
                    _t_ex.seconds//60, _t_ex.seconds % 60,
                ),
                    end="\r")

            # Calculate dot product of matrices
            cos_sim = datapoint[i*step_size:(i+1)*step_size] @ self.__dat.T

            # Get indices of highest value
            ind = np.argpartition(cos_sim, -self.k, axis=1)[:, -self.k:]

            # Get corresponding labels from internal data
            labels = np.array([self.__lab[ind[i]] for i in range(len(ind))])

            # Do majority vote for each input data point
            labels = np.array([np.argmax(np.bincount(labels[i]))
                              for i in range(len(labels))])

            labels_accumulator.append(labels)

        if self.__verbose:
            _en_t = datetime.datetime.now()
            _t = _en_t - _st_t
            _t_ex = _t

            print("Finsihed datapoints {}/{} ({}/{})    {}m{}s/{}m{}s".format(
                len(datapoint), len(datapoint),
                max_i, max_i,
                _t.seconds//60, _t.seconds % 60,
                _t_ex.seconds//60, _t_ex.seconds % 60,
            ),)

        return np.concatenate(labels_accumulator)


def calculate_accuraccy(true: np.ndarray, pred: np.ndarray) ->  float:
    """Calculates the accuraccy of a prediction in refference to the ground-truth

    Args:
        true (np.ndarray): Ground-truth of shape [N]
        pred (np.ndarray): Prediction of shape [N]

    Returns:
        float: The accuracy over all datasamples
    """
    assert true.shape == pred.shape

    p = true-pred
    wrong = np.count_nonzero(p)
    correct = len(true) - wrong

    return correct/len(true)* 100

def precission_recall(true: np.ndarray, pred: np.ndarray) ->  tuple:
    """Calculates the accuraccy of a prediction in refference to the ground-truth per class

    Args:
        true (np.ndarray): Ground-truth of shape [N]
        pred (np.ndarray): Prediction of shape [N]

    Returns:
        Tuple: precission and recall
    """
    
    precission = precision_score(true,pred, average = None)
    recall =        recall_score(true,pred, average = None)

    return (precission * 100,recall * 100)


print(calculate_accuraccy(
    np.array([0,1,1,2,2,2,2,2]),
    np.array([0,1,2,1,2,2,2,0])
      ))
print(precission_recall(
    np.array([0,1,1,2,2,2,2,2]),
    np.array([0,1,2,1,2,2,2,0])
      ))

62.5
(array([50., 50., 75.]), array([100.,  50.,  60.]))


In [3]:
if 0:
    dfs = preprocess_dataset("condensed", 10, False, persons=[1, 2], )


In [4]:
if 0:
    knn = KNN(len(dfs[list(dfs.keys())[0]].columns), 5)
    knn.shape
    for k in dfs.keys():
        knn.store_data(dfs[k])
        print(knn.shape)
    print("Training completed")
    
    
    dp = dfs["p1_d1"].to_numpy()[:1000,]

    knn.set_reduce_dimensions(5)
    knn.get_pca()
    
    knn.verbose()
    
    print("\n\nStart testing")
    y = knn(dp[:,:-1])
    print("Accuracy {}  Recall {}".format(*calculate_accuraccy(
        y, dp[:, -1])))


### Analyze performance
Show how good we can predict one day by the previous day per person

Problems: It takes too long because data is too high-dimensional $~(|4e5| \cdot |4e5|) = ~ 10^{11}$ matrix values that have to be computed

Ideas to improve performance:
- Use tensors on GPU → Not practicable with current memory usage

Ideas to reduce dimensionality: 
- 1. Make bigger time windows (Reduce rows) → From 10 (= 0.1s) to 3000 (=30s) "Reason: Non reading can also have short glimpses of reading. Like in original study"
- 2. Use PCA on stored data and input data (Reduce columns)

In [5]:
if 0:
    dfs = preprocess_dataset(
    "condensed", 3000, overlapping=False, persons=list(range(1, 11, 1)))


In [6]:
def overall_acc_for_k(_dfs, k):
    ov_acc = []

    for p in range(1,11, 1):
        df = _dfs["p{}_d2".format(p)]
        knn = KNN(len(df.columns), k)
        # print("")
        # knn.verbose()

        knn.store_data(_dfs["p{}_d1".format(p)])

        pred = knn(balance_class_distribution(df.to_numpy()[:, :-1]))
        acc = calculate_accuraccy(pred, df.to_numpy()[:, -1])

        #print("Accuraccy for p = {} is {:.2f}".format(p,acc))
        ov_acc.append(acc)


    print("\nOverall accuraccy in predicting the second day was {:.2f}({:.2f}) for K = {}".format(
        np.mean(ov_acc), np.std(ov_acc), k))
    
    return ov_acc


def log_and_plot_KNN(_dfs, name):
    log = pd.DataFrame()
    for K in [1, 2, 3, 5, 7, 10, 15]:
        acc = overall_acc_for_k(_dfs, K)
        
        log = pd.concat([log, pd.DataFrame({
            "K" : [str(K) for _ in range(len(acc))], "acc" : acc
        })])
        
    sns.boxplot(data=log, x="acc", y="K")

    sns.despine(trim = True)

    plt.grid(axis = "x", alpha = 0.6)
    plt.savefig("res/{}.png".format(name))
    plt.savefig("res/{}.pdf".format(name))
    plt.close()
    return log

For all k the accuraccy in predicting day 2 from day 1 per person is around 58% for $1<k\leq15$

In [7]:
if 1:
    log = log_and_plot_KNN(_dfs = preprocess_dataset(
    "condensed",normalize= True),
                name="KNN_norm")

    log = log_and_plot_KNN(_dfs = preprocess_dataset(
        "condensed", normalize= False),
                    name="KNN_raw")

    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed", normalize= True), num_pc=2),
                    name="KNN_norm_pca2")
    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed", normalize= True), num_pc=5),
                    name="KNN_norm_pca5")
    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed",normalize= True), num_pc=10),
                    name="KNN_norm_pca10")

    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed", normalize= False), num_pc=2),
                    name="KNN_raw_pca2")
    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed", normalize= False), num_pc=5),
                    name="KNN_raw_pca5")
    log = log_and_plot_KNN(_dfs = pca_dataset(
        preprocess_dataset(
        "condensed", normalize= False), num_pc=10),
                    name="KNN_raw_pca10")


Finished p1_d1. Took 0 minutes, 38 seconds
Finished p1_d2. Took 0 minutes, 47 seconds
Finished p2_d1. Took 0 minutes, 38 seconds
Finished p2_d2. Took 0 minutes, 32 seconds
Finished p3_d1. Took 0 minutes, 24 seconds
Finished p3_d2. Took 0 minutes, 24 seconds
Finished p4_d1. Took 0 minutes, 30 seconds
Finished p4_d2. Took 0 minutes, 30 seconds
Finished p5_d1. Took 0 minutes, 24 seconds
Finished p5_d2. Took 0 minutes, 13 seconds
Finished p6_d1. Took 0 minutes, 18 seconds
Finished p6_d2. Took 0 minutes, 18 seconds
Finished p7_d1. Took 0 minutes, 13 seconds
Finished p7_d2. Took 0 minutes, 14 seconds
Finished p8_d1. Took 0 minutes, 24 seconds
Finished p8_d2. Took 0 minutes, 27 seconds
Finished p9_d1. Took 0 minutes, 23 seconds
Finished p9_d2. Took 0 minutes, 12 seconds
Finished p10_d1. Took 0 minutes, 19 seconds
Finished p10_d2. Took 0 minutes, 17 seconds

Overall accuraccy in predicting the second day was 44.85(8.47) for K = 1

Overall accuraccy in predicting the second day was 49.85(7.71) 

| **Normalized** | **PCA** | **Best Acc**   | K        |
|----------------|---------|----------------|----------|
|        +       | None    | 58             | 7/**10**/15  |
|        +       | 2       | 62             | 7/10/**15**   |
|        +       | 5       | 57             | 7/10/15   |
|        +       | 10       | 57             | **2**/10/15   |
|        -       | None       | 59            | 2/10/**15**   |
|        -       | 2       | 60             | 2/5/7/**10**/15   |
|        -       | 5       | 58             | **2**   |
|        -       | 10       | 61             | **2**   |
